In [36]:
import onnx
import os
import wandb
import onnxruntime as ort
import numpy as np
from utils.utils import *
from test import test as evals


## Loading the model

In [ ]:

model = onnx.load(os.getcwd()+"/model.onnx")
(onnx.checker.check_model(model))

## See how the model is saved

In [47]:
print(model)

ir_version: 5
opset_import {
  version: 10
}
producer_name: "pytorch"
producer_version: "1.11.0"
graph {
  node {
    input: "input"
    input: "layer1.0.weight"
    input: "layer1.0.bias"
    output: "input.1"
    name: "Conv_0"
    op_type: "Conv"
    attribute {
      name: "dilations"
      type: INTS
      ints: 1
      ints: 1
    }
    attribute {
      name: "group"
      type: INT
      i: 1
    }
    attribute {
      name: "kernel_shape"
      type: INTS
      ints: 5
      ints: 5
    }
    attribute {
      name: "pads"
      type: INTS
      ints: 2
      ints: 2
      ints: 2
      ints: 2
    }
    attribute {
      name: "strides"
      type: INTS
      ints: 1
      ints: 1
    }
  }
  node {
    input: "input.1"
    output: "onnx::MaxPool_8"
    name: "Relu_1"
    op_type: "Relu"
  }
  node {
    input: "onnx::MaxPool_8"
    output: "input.4"
    name: "MaxPool_2"
    op_type: "MaxPool"
    attribute {
      name: "ceil_mode"
      type: INT
      i: 0
    }
    attr

## Recover Data as we had in the training to get the test

In [38]:
train, test = get_data(train=True), get_data(train=False)
test_loader = make_loader(test, batch_size=128)

## Using  the functionalitis fof the onnx

In [63]:
print(test_loader.dataset.dataset.data.float().unsqueeze(0).numpy().shape)

(1, 10000, 28, 28)


In [85]:
ort_sess = ort.InferenceSession(os.getcwd()+"/model.onnx")
ort_input = {ort_sess.get_inputs()[0].name: test_loader.dataset.dataset.data.float().unsqueeze(1).numpy()} # The unsqueeze because the session is expecting a batch dimension

# Making the run session
outputs = ort_sess.run(None, ort_input)[-1]
predicted, actual = outputs.argmax(1), test_loader.dataset.dataset.targets.numpy()


In [87]:
print("The accuracy is: ",np.sum(predicted == actual)/actual.shape[0])


The accuracy is:  0.9876
